In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [3]:
data_dir = '/content/drive/MyDrive/PLSU/'

In [4]:
img_size = (224, 224)
batch_size = 8

In [24]:
data_df = pd.read_excel(data_dir + 'label.xlsx', engine='openpyxl')
data_df

,File Number,life,head,heart,label
0,1,0,0,0,000
1,2,0,0,0,000
2,5,0,1,0,010
3,12,0,0,1,001
4,13,0,1,0,010
...,...,...,...,...,...
1031,5354,1,0,1,101
1032,5366,0,0,0,000
1033,5367,0,0,0,000
1034,5372,0,-,1,0-1


In [25]:
data_df = data_df[(data_df['life'] != '-') & (data_df['life'] != '')]
data_df = data_df[(data_df['head'] != '-') & (data_df['head'] != '')]
data_df = data_df[(data_df['heart'] != '-') & (data_df['heart'] != '')]
data_df

,File Number,life,head,heart,label
0,1,0,0,0,000
1,2,0,0,0,000
2,5,0,1,0,010
3,12,0,0,1,001
4,13,0,1,0,010
...,...,...,...,...,...
1030,5353,0,0,1,001
1031,5354,1,0,1,101
1032,5366,0,0,0,000
1033,5367,0,0,0,000


In [26]:
def reconstruct_file_name(file_number):
    return 'image' + str(file_number) + '.png'

data_df['file'] = data_df['File Number'].apply(reconstruct_file_name)
del data_df['File Number']

data_df

,life,head,heart,label,file
0,0,0,0,000,image1.png
1,0,0,0,000,image2.png
2,0,1,0,010,image5.png
3,0,0,1,001,image12.png
4,0,1,0,010,image13.png
...,...,...,...,...,...
1030,0,0,1,001,image5353.png
1031,1,0,1,101,image5354.png
1032,0,0,0,000,image5366.png
1033,0,0,0,000,image5367.png


In [27]:
print(data_df['life'].dtype)

int64


In [31]:
data_df = data_df.astype('string')
print(data_df['life'].dtype)

string


In [32]:
data_df['head']

0       0
1       0
2       1
3       0
4       1
       ..
1030    0
1031    0
1032    0
1033    0
1035    0
Name: head, Length: 996, dtype: string

In [40]:
train_df = data_df.sample(frac=0.7, random_state=42)
valid_df = data_df.drop(train_df.index).sample(frac=0.5, random_state=42)
test_df = data_df.drop(train_df.index).drop(valid_df.index)
print(len(train_df), len(valid_df), len(test_df))

697 150 149


In [41]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
)

In [42]:
data_generator = ImageDataGenerator(rescale=1./255)

## LIFE LINE

In [43]:
life_train_generator = data_generator.flow_from_dataframe(
    train_df,
    data_dir+'Mask',
    x_col='file',
    y_col='life',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

life_valid_generator = data_generator.flow_from_dataframe(
    valid_df,
    data_dir+'Mask',
    x_col='file',
    y_col='life',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

life_test_generator = data_generator.flow_from_dataframe(
    test_df,
    data_dir+'Mask',
    x_col='file',
    y_col='life',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

Found 697 validated image filenames belonging to 2 classes.
Found 150 validated image filenames belonging to 2 classes.
Found 149 validated image filenames belonging to 2 classes.


In [44]:
# CNN 모델 생성
life_model = models.Sequential()
life_model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 3)))
life_model.add(layers.MaxPooling2D((2, 2)))
life_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
life_model.add(layers.MaxPooling2D((2, 2)))
life_model.add(layers.Conv2D(128, (3, 3), activation='relu'))
life_model.add(layers.MaxPooling2D((2, 2)))
life_model.add(layers.Conv2D(256, (3, 3), activation='relu'))
life_model.add(layers.MaxPooling2D((2, 2)))
life_model.add(layers.Flatten())
life_model.add(layers.Dense(512, activation='relu'))
life_model.add(layers.Dropout(0.5))
life_model.add(layers.Dense(256, activation='relu'))
life_model.add(layers.Dropout(0.5))
life_model.add(layers.Dense(1, activation='sigmoid'))

In [45]:
# 모델 컴파일
life_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [47]:
# 모델 학습
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

life_model_checkpoint_path = data_dir + 'unseperated_life_model_checkpoint.h5'
early_stopping_patience = 5

early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=early_stopping_patience)
model_checkpoint = ModelCheckpoint(life_model_checkpoint_path, monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

life_history = life_model.fit(
    life_train_generator,
    steps_per_epoch=life_train_generator.samples // batch_size,
    epochs=30,
    validation_data=life_valid_generator,
    validation_steps=life_valid_generator.samples // batch_size,
    callbacks=[early_stopping, model_checkpoint]
)

Epoch 1/30
87/87 [==============================] - ETA: 0s - loss: 0.6252 - accuracy: 0.6473
Epoch 1: val_accuracy improved from -inf to 0.56944, saving model to /content/drive/MyDrive/PLSU/unseperated_life_model_checkpoint.h5
87/87 [==============================] - 21s 239ms/step - loss: 0.6252 - accuracy: 0.6473 - val_loss: 0.6616 - val_accuracy: 0.5694
Epoch 2/30
87/87 [==============================] - ETA: 0s - loss: 0.5908 - accuracy: 0.6502
Epoch 2: val_accuracy improved from 0.56944 to 0.57639, saving model to /content/drive/MyDrive/PLSU/unseperated_life_model_checkpoint.h5
87/87 [==============================] - 26s 301ms/step - loss: 0.5908 - accuracy: 0.6502 - val_loss: 0.6390 - val_accuracy: 0.5764
Epoch 3/30
87/87 [==============================] - ETA: 0s - loss: 0.5240 - accuracy: 0.6967
Epoch 3: val_accuracy improved from 0.57639 to 0.70833, saving model to /content/drive/MyDrive/PLSU/unseperated_life_model_checkpoint.h5
87/87 [==============================] - 28s 3

In [48]:
from tensorflow.keras.models import load_model

life_loaded_model = load_model(life_model_checkpoint_path)
loss, accuracy = life_loaded_model.evaluate(life_test_generator, steps=len(life_test_generator))

19/19 [==============================] - 35s 2s/step - loss: 0.6535 - accuracy: 0.8054


In [50]:
import numpy as np
from sklearn.metrics import f1_score

life_pred = life_loaded_model.predict(life_test_generator)
life_pred_classes = np.argmax(life_pred, axis=1)

life_true = life_test_generator.classes

life_f1 = f1_score(life_true, life_pred_classes, average='weighted')
print(life_f1)

19/19 [==============================] - 3s 176ms/step
0.5390690277890084


## HEAD LINE

In [51]:
head_train_generator = data_generator.flow_from_dataframe(
    train_df,
    data_dir+'Mask',
    x_col='file',
    y_col='head',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

head_valid_generator = data_generator.flow_from_dataframe(
    valid_df,
    data_dir+'Mask',
    x_col='file',
    y_col='head',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

head_test_generator = data_generator.flow_from_dataframe(
    test_df,
    data_dir+'Mask',
    x_col='file',
    y_col='head',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

Found 697 validated image filenames belonging to 2 classes.
Found 150 validated image filenames belonging to 2 classes.
Found 149 validated image filenames belonging to 2 classes.


In [52]:
# CNN 모델 생성
head_model = models.Sequential()
head_model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 3)))
head_model.add(layers.MaxPooling2D((2, 2)))
head_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
head_model.add(layers.MaxPooling2D((2, 2)))
head_model.add(layers.Conv2D(128, (3, 3), activation='relu'))
head_model.add(layers.MaxPooling2D((2, 2)))
head_model.add(layers.Conv2D(256, (3, 3), activation='relu'))
head_model.add(layers.MaxPooling2D((2, 2)))
head_model.add(layers.Flatten())
head_model.add(layers.Dense(512, activation='relu'))
head_model.add(layers.Dropout(0.5))
head_model.add(layers.Dense(256, activation='relu'))
head_model.add(layers.Dropout(0.5))
head_model.add(layers.Dense(1, activation='sigmoid'))

In [53]:
# 모델 컴파일
head_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [54]:
# 모델 학습
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

head_model_checkpoint_path = data_dir + 'unseperated_head_model_checkpoint.h5'
early_stopping_patience = 5

early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=early_stopping_patience)
model_checkpoint = ModelCheckpoint(head_model_checkpoint_path, monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

head_history = head_model.fit(
    head_train_generator,
    steps_per_epoch=head_train_generator.samples // batch_size,
    epochs=30,
    validation_data=head_valid_generator,
    validation_steps=head_valid_generator.samples // batch_size,
    callbacks=[early_stopping, model_checkpoint]
)

Epoch 1/30
87/87 [==============================] - ETA: 0s - loss: 0.7232 - accuracy: 0.6734
Epoch 1: val_accuracy improved from -inf to 0.70833, saving model to /content/drive/MyDrive/PLSU/unseperated_head_model_checkpoint.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


87/87 [==============================] - 24s 257ms/step - loss: 0.7232 - accuracy: 0.6734 - val_loss: 0.6052 - val_accuracy: 0.7083
Epoch 2/30
87/87 [==============================] - ETA: 0s - loss: 0.6227 - accuracy: 0.6836
Epoch 2: val_accuracy did not improve from 0.70833
87/87 [==============================] - 19s 222ms/step - loss: 0.6227 - accuracy: 0.6836 - val_loss: 0.6092 - val_accuracy: 0.7014
Epoch 3/30
87/87 [==============================] - ETA: 0s - loss: 0.6319 - accuracy: 0.6865
Epoch 3: val_accuracy improved from 0.70833 to 0.71528, saving model to /content/drive/MyDrive/PLSU/unseperated_head_model_checkpoint.h5
87/87 [==============================] - 22s 259ms/step - loss: 0.6319 - accuracy: 0.6865 - val_loss: 0.5703 - val_accuracy: 0.7153
Epoch 4/30
87/87 [==============================] - ETA: 0s - loss: 0.6078 - accuracy: 0.6763
Epoch 4: val_accuracy did not improve from 0.71528
87/87 [==============================] - 21s 246ms/step - loss: 0.6078 - accuracy: 

In [55]:
from tensorflow.keras.models import load_model

head_loaded_model = load_model(head_model_checkpoint_path)
loss, accuracy = head_loaded_model.evaluate(head_test_generator, steps=len(head_test_generator))

19/19 [==============================] - 8s 421ms/step - loss: 0.4485 - accuracy: 0.9128


In [57]:
import numpy as np
from sklearn.metrics import f1_score

head_pred = head_loaded_model.predict(head_test_generator)
head_pred_classes = np.argmax(head_pred, axis=1)

head_true = head_test_generator.classes

head_f1 = f1_score(head_true, head_pred_classes, average='weighted')
print(head_f1)

19/19 [==============================] - 4s 187ms/step
0.5738387670106375


## HEART LINE

In [58]:
heart_train_generator = data_generator.flow_from_dataframe(
    train_df,
    data_dir+'Mask',
    x_col='file',
    y_col='heart',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

heart_valid_generator = data_generator.flow_from_dataframe(
    valid_df,
    data_dir+'Mask',
    x_col='file',
    y_col='heart',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

heart_test_generator = data_generator.flow_from_dataframe(
    test_df,
    data_dir+'Mask',
    x_col='file',
    y_col='heart',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

Found 697 validated image filenames belonging to 2 classes.
Found 150 validated image filenames belonging to 2 classes.
Found 149 validated image filenames belonging to 2 classes.


In [59]:
# CNN 모델 생성
heart_model = models.Sequential()
heart_model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 3)))
heart_model.add(layers.MaxPooling2D((2, 2)))
heart_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
heart_model.add(layers.MaxPooling2D((2, 2)))
heart_model.add(layers.Conv2D(128, (3, 3), activation='relu'))
heart_model.add(layers.MaxPooling2D((2, 2)))
heart_model.add(layers.Conv2D(256, (3, 3), activation='relu'))
heart_model.add(layers.MaxPooling2D((2, 2)))
heart_model.add(layers.Flatten())
heart_model.add(layers.Dense(512, activation='relu'))
heart_model.add(layers.Dropout(0.5))
heart_model.add(layers.Dense(256, activation='relu'))
heart_model.add(layers.Dropout(0.5))
heart_model.add(layers.Dense(1, activation='sigmoid'))

In [60]:
# 모델 컴파일
heart_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [61]:
# 모델 학습
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

heart_model_checkpoint_path = data_dir + 'unseperated_heart_model_checkpoint.h5'
early_stopping_patience = 5

early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=early_stopping_patience)
model_checkpoint = ModelCheckpoint(heart_model_checkpoint_path, monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

heart_history = heart_model.fit(
    heart_train_generator,
    steps_per_epoch=heart_train_generator.samples // batch_size,
    epochs=30,
    validation_data=heart_valid_generator,
    validation_steps=heart_valid_generator.samples // batch_size,
    callbacks=[early_stopping, model_checkpoint]
)

Epoch 1/30
87/87 [==============================] - ETA: 0s - loss: 0.6973 - accuracy: 0.5109
Epoch 1: val_accuracy improved from -inf to 0.55556, saving model to /content/drive/MyDrive/PLSU/unseperated_heart_model_checkpoint.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


87/87 [==============================] - 24s 255ms/step - loss: 0.6973 - accuracy: 0.5109 - val_loss: 0.6922 - val_accuracy: 0.5556
Epoch 2/30
87/87 [==============================] - ETA: 0s - loss: 0.6909 - accuracy: 0.5356
Epoch 2: val_accuracy improved from 0.55556 to 0.57639, saving model to /content/drive/MyDrive/PLSU/unseperated_heart_model_checkpoint.h5
87/87 [==============================] - 25s 283ms/step - loss: 0.6909 - accuracy: 0.5356 - val_loss: 0.6722 - val_accuracy: 0.5764
Epoch 3/30
87/87 [==============================] - ETA: 0s - loss: 0.6844 - accuracy: 0.5486
Epoch 3: val_accuracy did not improve from 0.57639
87/87 [==============================] - 20s 230ms/step - loss: 0.6844 - accuracy: 0.5486 - val_loss: 0.6495 - val_accuracy: 0.5764
Epoch 4/30
87/87 [==============================] - ETA: 0s - loss: 0.6430 - accuracy: 0.6328
Epoch 4: val_accuracy improved from 0.57639 to 0.65278, saving model to /content/drive/MyDrive/PLSU/unseperated_heart_model_checkpoin

In [62]:
from tensorflow.keras.models import load_model

heart_loaded_model = load_model(heart_model_checkpoint_path)
loss, accuracy = heart_loaded_model.evaluate(heart_test_generator, steps=len(heart_test_generator))

19/19 [==============================] - 3s 161ms/step - loss: 0.6333 - accuracy: 0.7584


In [63]:
import numpy as np
from sklearn.metrics import f1_score

heart_pred = heart_loaded_model.predict(heart_test_generator)
heart_pred_classes = np.argmax(heart_pred, axis=1)

heart_true = heart_test_generator.classes

heart_f1 = f1_score(heart_true, heart_pred_classes, average='weighted')
print(heart_f1)

19/19 [==============================] - 3s 164ms/step
0.2650065859624914
